In [2]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

/kaggle/input
/kaggle/input/kitte-dataset
/kaggle/input/kitte-dataset/training
/kaggle/input/kitte-dataset/training/label_2
/kaggle/input/kitte-dataset/training/image_2


In [3]:
import os
import shutil
import cv2
import yaml
from sklearn.model_selection import train_test_split

# This script does the following:

* Converts the KITTI annotations to YOLO format.
* Splits the dataset into training, validation, and test sets.
* Moves the files to the appropriate directories.
* Creates a data.yaml file required for training YOLO models.

In [4]:

# Define paths
kitti_base_path = '/kaggle/input/kitte-dataset/training'
yolo_base_path = '/kaggle/working/yolo'  # Use a writable directory
images_path = os.path.join(kitti_base_path, 'image_2')
labels_path = os.path.join(kitti_base_path, 'label_2')
yolo_images_path = os.path.join(yolo_base_path, 'images')
yolo_labels_path = os.path.join(yolo_base_path, 'labels')

# Create YOLO directories
os.makedirs(yolo_images_path, exist_ok=True)
os.makedirs(yolo_labels_path, exist_ok=True)

kitti_classes = ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']

# Define function to convert KITTI bbox to YOLO bbox
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

# Process each label file
for label_file in os.listdir(labels_path):
    if not label_file.endswith('.txt'):
        continue
    image_file = label_file.replace('.txt', '.png')

    image = cv2.imread(os.path.join(images_path, image_file))
    h, w, _ = image.shape

    with open(os.path.join(labels_path, label_file), 'r') as lf:
        lines = lf.readlines()
    yolo_labels = []
    for line in lines:
        elements = line.strip().split(' ')
        class_id = elements[0]
        if class_id in kitti_classes:
            class_id = kitti_classes.index(class_id)  # In KITTI, the first element is the class
            xmin, ymin, xmax, ymax = map(float, elements[4:8])
            bbox = convert_bbox((w, h), (xmin, xmax, ymin, ymax))
            yolo_labels.append(f"{class_id} {' '.join(map(str, bbox))}\n")

    with open(os.path.join(yolo_labels_path, label_file), 'w') as yf:
        yf.writelines(yolo_labels)
    # Copy image to YOLO directory
    shutil.copy(os.path.join(images_path, image_file), yolo_images_path)

# Split dataset into train, val, test sets
all_images = [f for f in os.listdir(yolo_images_path) if f.endswith('.png')]
train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)
val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)

# Function to move files to appropriate directories
def move_files(file_list, dest_dir):
    os.makedirs(os.path.join(dest_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dest_dir, 'labels'), exist_ok=True)
    for file_name in file_list:
        shutil.move(os.path.join(yolo_images_path, file_name), os.path.join(dest_dir, 'images', file_name))
        label_file = file_name.replace('.png', '.txt')
        shutil.move(os.path.join(yolo_labels_path, label_file), os.path.join(dest_dir, 'labels', label_file))

# Move files to train, val, test directories
move_files(train_images, os.path.join(yolo_base_path, 'train'))
move_files(val_images, os.path.join(yolo_base_path, 'val'))
move_files(test_images, os.path.join(yolo_base_path, 'test'))

print("Conversion and splitting complete.")



# Define the data dictionary
data = {
    'train': '/kaggle/working/yolo/train/images',
    'val': '/kaggle/working/yolo/val/images',
    'test': '/kaggle/working/yolo/test/images',
    'nc': 8,  # number of classes
    'names': ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']
}

# Save to data.yaml
with open('/kaggle/working/yolo/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

print("data.yaml file created successfully.")


Conversion and splitting complete.
data.yaml file created successfully.


In [5]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 22), reused 27 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 33.45 MiB/s, done.
Resolving deltas: 100% (11712/11712), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 58.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.3
    Uninstalling setuptools-69.0.3:
      Successfully uninstalled setuptools-69.0.3
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Succ

In [6]:
!python train.py --img 640 --batch 16 --epochs 50 --data /kaggle/working/yolo/data.yaml --weights yolov5s.pt --cache


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-11-26 19:03:03.540912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 19:03:03.541038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 19:03:03.683141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to regi